<a href="https://colab.research.google.com/github/sidratulmuntahasara/CapX-AI-Agent/blob/main/CapxAIAgentTutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cell 1: Imports and Setup

In [11]:
!pip install -U google-generativeai

In [12]:
import time
from typing import Optional, Dict, List, Any
import requests
import http.client
import json
import google.generativeai as genai
from IPython.display import display, Markdown


API_KEY_COINGECKO = ''
API_KEY_SERPER = ''
API_KEY_GOOGLE_AI = ''

genai.configure(api_key=API_KEY_GOOGLE_AI)

# Cell 2: CoinGecko & Helper Functions

In [13]:
def get_json_data(url: str) -> dict:
    headers = {'x-cg-demo-api-key': API_KEY_COINGECKO}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Error fetching data: {response.status_code}")

def rate_limit(response: dict) -> bool:
    if "status" in response:
        if response["status"].get("error_code") == 429:
            print(f"Rate limit error: {response['status']['error_message']}")
            return True
    return False

def get_coin_by_id(id: str) -> dict:
    data = get_json_data(f"https://api.coingecko.com/api/v3/coins/{id}")
    if rate_limit(data):
        raise Exception("Rate limit reached")
    return data

def get_coins_markets(currency: str = "usd", order: str = "market_cap_desc", per_page: int = 250, page: int = 1) -> List[dict]:
    data = get_json_data(
        f"https://api.coingecko.com/api/v3/coins/markets?vs_currency={currency}&order={order}&per_page={per_page}&page={page}&sparkline=false"
    )
    if rate_limit(data):
        raise Exception("Rate limit reached")
    return data

def get_detailed_coin_info(ticker: str) -> Optional[Dict]:
    try:
        coins_list = get_coins_markets()
        coin_data = next((coin for coin in coins_list if coin['symbol'].upper() == ticker.upper()), None)

        if coin_data:
            coin_id = coin_data['id']
            coin_dict = get_coin_by_id(coin_id)

            if coin_dict:
                return {
                    "id": coin_id,
                    "symbol": coin_dict["symbol"],
                    "name": coin_dict["name"],
                    "price": coin_dict["market_data"]["current_price"].get("usd", 0),
                    "market_cap_rank": coin_dict["market_cap_rank"],
                    "description": coin_dict["description"].get("en", ""),
                }
    except Exception as e:
        print(f"Error in API call for {ticker}: {e}")
    return None

def get_top_coins(n: int = 100) -> List[Dict]:
    coins = get_coins_markets(per_page=n)
    return [{
        "rank": coin["market_cap_rank"],
        "symbol": coin["symbol"],
        "name": coin["name"],
        "price": coin["current_price"],
        "market_cap": coin["market_cap"],
        "volume_24h": coin["total_volume"],
        "change_24h": coin["price_change_percentage_24h"],
    } for coin in coins]


Note: These helper functions are used to interact with the CoinGecko API.
They handle JSON data retrieval, specific coin data fetching, and rate limiting.

# Search Tool

In [14]:
def search_coin(coin_name: str) -> Dict[str, Any]:
    conn = http.client.HTTPSConnection("google.serper.dev")
    payload = json.dumps({
        "q": f"{coin_name} cryptocurrency analysis"
    })
    headers = {
        'X-API-KEY': API_KEY_SERPER,
        'Content-Type': 'application/json'
    }
    conn.request("POST", "/search", payload, headers)
    res = conn.getresponse()
    data = res.read()
    return json.loads(data.decode("utf-8"))


#Using Gemini Flash

In [15]:
def analyze_coin(coin_data: Dict[str, Any], search_results: Dict[str, Any]) -> str:
    model = genai.GenerativeModel("gemini-1.5-flash")

    prompt = f"""Analyze this cryptocurrency data and search results. Provide a general report, recent trends, and potential alpha:

Coin Data:
{json.dumps(coin_data, indent=2)}

Search Results:
{json.dumps(search_results, indent=2)}"""

    response = model.generate_content(prompt)
    return response.text

#Agent Workflow

In [16]:
def crypto_analysis(coin_name: str) -> Dict[str, Any]:
    # Step 1: Get detailed coin info from CoinGecko
    time.sleep(6)  # Rate limiting
    coin_data = get_detailed_coin_info(coin_name)
    if not coin_data:
        return {"error": f"Unable to fetch data for {coin_name}"}

    # Step 2: Perform web search for recent news and analysis
    search_results = search_coin(coin_data['name'])

    # Step 3: Get top 10 coins for market context
    top_coins = get_top_coins(10)

    # Step 4: Analyze data with Google AI
    analysis = analyze_coin(coin_data, search_results)

    # Step 5: Compile all information into a comprehensive report
    report = {
        "coin_data": coin_data,
        "market_context": {
            "top_10_coins": top_coins,
            "coin_rank": next((coin['rank'] for coin in top_coins if coin['symbol'] == coin_name.lower()), "Not in top 10")
        },
        "recent_news": search_results,
        "ai_analysis": analysis
    }

    return report

# Report Generation


---



In [17]:
def generate_markdown_report(report: Dict[str, Any]) -> str:
    coin_data = report.get('coin_data', {})
    market_context = report.get('market_context', {})

    md_content = f"""
# Cryptocurrency Analysis Report for {coin_data.get('name', 'Unknown')}

## Coin Data
- Name: {coin_data.get('name', 'Unknown')}
- Symbol: {coin_data.get('symbol', 'Unknown')}
- Current Price: ${coin_data.get('price', 'N/A'):,.2f}
- Market Cap Rank: {coin_data.get('market_cap_rank', 'N/A')}


## Description
{coin_data.get('description', 'No description available.')}

## Market Context
### Top 10 Coins:
{chr(10).join([f"{i+1}. {coin.get('name', 'Unknown')} ({coin.get('symbol', 'Unknown')})" for i, coin in enumerate(market_context.get('top_10_coins', []))])}

## AI Analysis
{report.get('ai_analysis', 'No AI analysis available')}
    """
    return md_content

# Main function to run the analysis


In [19]:
def main():
    coin_name = input("Enter the name of the cryptocurrency to analyze: ")
    print(f"Analyzing {coin_name}...")

    try:
        analysis_report = crypto_analysis(coin_name)
        if "error" in analysis_report:
            print(f"Error: {analysis_report['error']}")
        else:
            md_report = generate_markdown_report(analysis_report)

            # Display in Jupyter Notebook
            display(Markdown(md_report))

            # Save to file
            with open(f"{coin_name}_analysis.md", "w") as f:
                f.write(md_report)
            print(f"Analysis report saved as {coin_name}_analysis.md")
    except Exception as e:

        print(f"An error occurred: {e}")

# Cell 8: Run the main function
if __name__ == "__main__":
    main()

Enter the name of the cryptocurrency to analyze: nft
Analyzing nft...



# Cryptocurrency Analysis Report for APENFT

## Coin Data
- Name: APENFT
- Symbol: nft
- Current Price: $0.00
- Market Cap Rank: 150


## Description
APENFT Fund was born with the mission to register world-class artworks as NFTs on-chain. It is built on top of TRON, one of the world's top three public chains, and is powered by the world's largest distributed data storage system BitTorrent.

Like how blockchain democratizes finance like never before, APENFT, by turning top artists and art pieces into NFTs, not only upgrades the way artworks are hosted but also transforms them from being elite-exclusive items to something that truly belongs to the people and mirrors their aspirations. APENFT is the art for everyone.

## Market Context
### Top 10 Coins:
1. Bitcoin (btc)
2. Ethereum (eth)
3. Tether (usdt)
4. BNB (bnb)
5. Solana (sol)
6. USDC (usdc)
7. XRP (xrp)
8. Lido Staked Ether (steth)
9. Toncoin (ton)
10. Dogecoin (doge)

## AI Analysis
## APENFT (NFT) Cryptocurrency Report

**General Overview:**

APENFT is a cryptocurrency built on the TRON blockchain, focused on registering and securing world-class artworks as NFTs. It aims to democratize art by making it accessible to everyone through blockchain technology.

**Coin Data:**

* **Symbol:** NFT
* **Price:** $0.0000003819 (as of 2023-11-24)
* **Market Cap Rank:** 150
* **Description:**  APENFT aims to revolutionize art ownership by registering artworks as NFTs on the TRON blockchain, making them accessible and secure.

**Recent Trends:**

* **Search Interest:** There is strong interest in APENFT, evidenced by the numerous search results.  Users are particularly interested in price predictions and potential investment opportunities.
* **Price Predictions:**  Most price predictions for APENFT range from moderate growth to significant increases in the future. While there's no guarantee of these predictions coming true, they indicate a level of optimism regarding the project's potential.
* **Market Sentiment:**  The search results show a mix of positive and neutral sentiment. Some sources believe APENFT has the potential to grow substantially, while others point to the volatility of the crypto market.

**Potential Alpha:**

* **Adoption:**  The success of APENFT heavily depends on its ability to attract artists and collectors to its platform. Widespread adoption could lead to increased value for the NFT token.
* **TRON Blockchain:** APENFT leverages the TRON blockchain, which has a strong community and growing adoption. This could contribute to the project's success.
* **NFT Market:**  The NFT market is constantly evolving and growing, offering potential for APENFT to capitalize on its position as an NFT-focused platform.
* **Partnerships:**  APENFT has partnerships with notable organizations like the BitTorrent network, which could boost its visibility and credibility.

**Potential Risks:**

* **Market Volatility:** The cryptocurrency market is notoriously volatile, and APENFT's price could fluctuate significantly.
* **Competition:**  APENFT faces competition from other NFT platforms and marketplaces.
* **Regulatory Uncertainty:** The regulatory landscape for cryptocurrencies is still evolving, and changes could impact the project's future.

**Conclusion:**

APENFT presents an interesting opportunity within the burgeoning NFT market. It benefits from its association with the TRON blockchain and its focus on democratizing art ownership.  However, investors need to be aware of the risks associated with cryptocurrencies, including volatility, competition, and regulatory uncertainty. It's crucial to conduct thorough research and understand the project's fundamentals before investing in APENFT.

    

Analysis report saved as nft_analysis.md
